In [ ]:
# import the libraries
import numpy as np
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from itertools import product
import matplotlib.pyplot as plt
# from fitter import Fitter, get_common_distributions, get_distributions

In [ ]:
# import, shuffle, and see the data
ddf = dd.read_csv('/kaggle/input/stanford-ribonanza-rna-folding/train_data_p_unp.csv')
shfl_ddf = ddf.sample(frac = 1, random_state = 42)
shfl_ddf.head()

In [ ]:
dms_ddf = ddf.loc[ddf['experiment_type'] == "DMS_MaP"]
twoa3_ddf = ddf.loc[ddf['experiment_type'] == "2A3_MaP"]
dms_ddf.head()

In [ ]:
# Get the shape of the Dask DataFrame
shape = dms_ddf.shape

# Extract the number of rows and columns from the shape tuple
num_rows, num_columns = shape[0].compute(), shape[1]

# Print the size of the Dask DataFrame
print(f"Number of Rows: {num_rows}")
print(f"Number of Columns: {num_columns}")

In [ ]:
# Modified version to account for probability of unpaired bases from secondary structure predictor
bases={'A':0, 'C':1, 'G':2, 'U':3 }

def one_hot(string, p_unpaired):

    res = np.zeros((5, 206), # Now there are 5 rows in the input vector, 457 is maximum length
                   dtype=np.float32)
    res[4, :] = 1

    for j in range(len(string)):
        if string[j] in bases: # bases can be 'N' signifying missing: this corresponds to all 0 in the encoding
            res[ bases[ string[j] ], j ]= 1.
        res[4, j] = p_unpaired[j]

    return res

In [ ]:
# For p_unpaired data (changed the yield output)
import torch
import torch.nn as nn
import torch.nn.functional as F

class BedPeaksDataset(torch.utils.data.IterableDataset):

    def __init__(self, seq, p_unpaired, reactivities):
        super(BedPeaksDataset, self).__init__()
        self.seq = seq
        self.reactivities = reactivities
        self.p_unpaired = p_unpaired

    def __iter__(self):
        for i in range(len(self.seq)):
            yield(one_hot(self.seq[i], self.p_unpaired[i]), self.reactivities[i]) # positive example

# train_dataset = BedPeaksDataset(seqs, reactivities)
# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=10, num_workers = 0)

In [ ]:
class CNN_1d(nn.Module):

    def __init__(self,
                 n_output_channels = 206,# Normally 206, vs 457
                 filter_widths = [21, 15, 11, 7],
                 num_chunks = 5,
                 #max_pool_factor = 2,
                 nchannels = [5, 32, 64, 128, 256],
                 n_hidden = 500,
                 dropout = 0.2):

        super(CNN_1d, self).__init__()
        self.rf = 0 # running estimate of the receptive field
        self.chunk_size = 1 # running estimate of num basepairs corresponding to one position after convolutions

        conv_layers = []
        for i in range(len(nchannels)-1):
            conv_layers += [ nn.Conv1d(nchannels[i], nchannels[i+1], filter_widths[i], padding = 0, stride = 2),
                        nn.BatchNorm1d(nchannels[i+1]), # tends to help give faster convergence: https://arxiv.org/abs/1502.03167
                        nn.Dropout2d(dropout), # popular form of regularization: https://jmlr.org/papers/v15/srivastava14a.html
                        #nn.MaxPool1d(max_pool_factor),
                        nn.ELU(inplace=True)  ] # popular alternative to ReLU: https://arxiv.org/abs/1511.07289
            assert(filter_widths[i] % 2 == 1) # assume this
            self.rf += (filter_widths[i] - 1) * self.chunk_size
            #self.chunk_size *= max_pool_factor
        # If you have a model with lots of layers, you can create a list first and
        # then use the * operator to expand the list into positional arguments, like this:
        self.conv_net = nn.Sequential(*conv_layers)

        # Calculate the output size after convolutions and pooling
        total_length = 206  # Assuming the width of the input matrix is 206
        for filter_width in filter_widths:
            total_length = (total_length - filter_width + 1) # // max_pool_factor

        # Calculate the correct number of features to pass to the first linear layer
        conv_output_features = nchannels[-1] * total_length
        override = 1280
        self.dense_net = nn.Sequential(nn.Linear(override, n_hidden), # override = conv_output_features
                                        nn.BatchNorm1d(n_hidden),
                                        nn.Dropout(dropout),
                                        nn.ELU(inplace=True),
                                        nn.Linear(n_hidden, n_output_channels))

#         self.conv_net = nn.Sequential(*conv_layers)

#         self.seq_len = num_chunks * self.chunk_size + self.rf # amount of sequence context required

#         print("Receptive field:", self.rf, "Chunk size:", self.chunk_size, "Number chunks:", num_chunks)

#         self.dense_net = nn.Sequential( nn.Linear(nchannels[-1] * num_chunks, n_hidden),
#                                         nn.Dropout(dropout),
#                                         nn.ELU(inplace=True),
#                                         nn.Linear(n_hidden, n_output_channels) )

    def forward(self, x):
        net = self.conv_net(x)
        net = net.view(net.size(0), -1)
        net = self.dense_net(net)
        return(net)

In [ ]:
CNN_1d()

In [ ]:
def run_one_epoch(train_flag, dataloader, cnn_1d, optimizer, device="cuda"):

    torch.set_grad_enabled(train_flag)
    cnn_1d.train() if train_flag else cnn_1d.eval()

    losses = []
    accuracies = []

    for (x,y) in dataloader: # collection of tuples with iterator
        x = x.float()
        y = y.float()
        (x, y) = ( x.to(device), y.to(device) ) # transfer data to GPU

        output = cnn_1d(x) # forward pass
        output = output.squeeze() # remove spurious channel dimension
        loss = F.mse_loss(output, y).float()

        if train_flag:
            loss.backward() # back propagation
            optimizer.step()
            optimizer.zero_grad()

        losses.append(loss.detach().cpu().numpy())
        # accuracy = torch.mean( ( (output > 0.0) == (y > 0.5) ).float() ) # output is in logit space so threshold is 0.
        # accuracies.append(accuracy.detach().cpu().numpy())

    return( np.mean(losses))

In [ ]:
def train_model(cnn_1d, train_data, validation_data, epochs=100, patience=10, verbose = True): # lr = 0.001, weight_decay = 0
    """
    Train a 1D CNN model and record accuracy metrics.
    """
    # Move the model to the GPU here to make it runs there, and set "device" as above
    # TODO CODE
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cnn_1d.to(device)

    # 1. Make new BedPeakDataset and DataLoader objects for both training and validation data.
    # TODO CODE
    #train_dataset = BedPeaksDataset(train_data, cnn_1d.seq_len)
    #train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=10, num_workers = 0)
    #validation_dataset = BedPeaksDataset(validation_data, cnn_1d.seq_len)
    #validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=1000)

    # 2. Instantiates an optimizer for the model.
    # TODO CODE
    optimizer = torch.optim.Adam(cnn_1d.parameters(), amsgrad=True) # lr = lr weight_decay = weight_decay

    # 3. Run the training loop with early stopping.
    # TODO CODE
    train_losses = []
    validation_losses = []
    patience_counter = patience
    best_test_loss = np.inf
    check_point_filename = 'cnn_1d_checkpoint.pt' # to save the best model fit to date
    for epoch in range(epochs):
        start_time = timeit.default_timer()
        train_loss = run_one_epoch(True, train_dataloader, cnn_1d, optimizer, device)
        validation_loss = run_one_epoch(False, validation_dataloader, cnn_1d, optimizer, device)
        train_losses.append(train_loss)
        validation_losses.append(validation_loss)
        # train_accs.append(train_acc)
        # val_accs.append(val_acc)
        if validation_loss < best_test_loss:
            torch.save(cnn_1d.state_dict(), check_point_filename)
            best_test_loss = validation_loss
            patience_counter = patience
        else:
            patience_counter -= 1
            if patience_counter <= 0:
                cnn_1d.load_state_dict(torch.load(check_point_filename)) # recover the best model so far
                break
        elapsed = float(timeit.default_timer() - start_time)
        print("Epoch {} took {:.2f}s. Train loss: {:.4f}., Test loss: {:.4f}. Patience: {}".format(epoch+1, elapsed, train_loss, validation_loss, patience))

    # 4. Return the fitted model (not strictly necessary since this happens "in place"), train and validation accuracies.
    # TODO CODE
    return(cnn_1d, train_losses, validation_losses)

In [ ]:
pip install dask_ml

In [ ]:
# apply SN-filter (with p_unpaired calculation)
from dask_ml.model_selection import train_test_split
df_sn = ddf[ddf["SN_filter"]==1]

# split into 2A3 MaP and DMS MaP datasets
df_2A3 = df_sn[df_sn["experiment_type"]=="2A3_MaP"]
df_DMS = df_sn[df_sn["experiment_type"]=="DMS_MaP"]

# split into train and test
X_2A3_seq = df_2A3["sequence"]
X_2A3_p_unpaired = df_2A3.loc[:, "p_unp_1":"p_unp_206"]
y_2A3 = df_2A3.loc[:, df_2A3.columns.str.fullmatch("reactivity_\d\d\d\d")]
X_2A3_train_seq, X_2A3_test_seq, X_2A3_train_p_unpaired, X_2A3_test_p_unpaired, y_2A3_train, y_2A3_test = train_test_split(X_2A3_seq, X_2A3_p_unpaired, y_2A3, test_size=0.2, shuffle=True, blockwise=True, random_state=42)
X_2A3_train_seq, X_2A3_validation_seq, X_2A3_train_p_unpaired, X_2A3_validation_p_unpaired, y_2A3_train, y_2A3_validation = train_test_split(X_2A3_train_seq, X_2A3_train_p_unpaired, y_2A3_train, test_size=0.25, shuffle=True, blockwise=True, random_state=42)

X_DMS = df_DMS["sequence"]
y_DMS = df_DMS.loc[:, df_DMS.columns.str.fullmatch("reactivity_\d\d\d\d")]
X_DMS_train, X_DMS_test, y_DMS_train, y_DMS_test = train_test_split(X_DMS, y_DMS, test_size=0.2, shuffle=True, blockwise=True, random_state=42)
X_DMS_train, X_DMS_validation, y_DMS_train, y_DMS_validation = train_test_split(X_DMS_train, y_DMS_train, test_size=0.25, shuffle=True, blockwise=True, random_state=42)

In [ ]:
def df_toArray(ddf1A, ddf1B, ddf2): # for sequence, p_unpaired, and reactivity
    subset_columns = []
    for i in range(206):
        subset_columns.append("reactivity_0"+str(i+1).zfill(3))

    # Use .to_dask_array() to convert the subset of the DataFrame to a Dask Array
    # subset = small_set[subset_columns]

    # Compute the subset of the Dask DataFrame and convert it to a Pandas DataFrame
    reactivities = ddf2.compute().to_numpy()
    p_unpaired = ddf1B.compute().to_numpy()

    row_means = np.nanmean(reactivities, axis=1)

    # Iterate over each element and replace NaN with the row mean
    for i, row in enumerate(reactivities):
        mask = np.isnan(row)
        reactivities[i, mask] = row_means[i]

    reactivities

    seqs = ddf1A.compute().tolist()

    return seqs, p_unpaired, reactivities

# df_toArray(X_2A3_train, y_2A3_train)

In [ ]:
def df_toArray_test(ddf1A, ddf1B, ddf2): # for sequence, p_unpaired, and reactivity
    subset_columns = []
    for i in range(206):
        subset_columns.append("reactivity_0"+str(i+1).zfill(3))

    # Use .to_dask_array() to convert the subset of the DataFrame to a Dask Array
    # subset = small_set[subset_columns]

    # Compute the subset of the Dask DataFrame and convert it to a Pandas DataFrame
    reactivities = ddf2.compute().to_numpy()
    p_unpaired = ddf1B.compute().to_numpy()

    row_means = np.nanmean(reactivities, axis=1)

    # Iterate over each element and replace NaN with the row mean
    for i, row in enumerate(reactivities):
        mask = np.isnan(row)
        reactivities[i, mask] = row_means[i]

    reactivities

    seqs = ddf1A.compute().tolist()

    return seqs, p_unpaired, reactivities

# df_toArray(X_2A3_train, y_2A3_train)

In [ ]:
# For p_unpaired, 2A3

seqs, p_unpaired, reactivities = df_toArray(X_2A3_train_seq, X_2A3_train_p_unpaired, y_2A3_train)
train_dataset = BedPeaksDataset(seqs, p_unpaired, reactivities)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1000, num_workers = 0)

seqs, p_unpaired, reactivities = df_toArray(X_2A3_validation_seq, X_2A3_validation_p_unpaired, y_2A3_validation)
validation_dataset = BedPeaksDataset(seqs, p_unpaired, reactivities)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=1000, num_workers = 0)

seqs, p_unpaired, reactivities = df_toArray(X_2A3_test_seq, X_2A3_test_p_unpaired, y_2A3_test)
test_dataset = BedPeaksDataset(seqs, p_unpaired, reactivities)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, num_workers = 0)

In [ ]:
# For p_unpaired, DMS

seqs, p_unpaired, reactivities = df_toArray(X_DMS_train_seq, X_DMS_train_p_unpaired, y_DMS_train)
train_dataset = BedPeaksDataset(seqs, p_unpaired, reactivities)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1000, num_workers = 0)

seqs, p_unpaired, reactivities = df_toArray(X_DMS_validation_seq, X_DMS_validation_p_unpaired, y_DMS_validation)
validation_dataset = BedPeaksDataset(seqs, p_unpaired, reactivities)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=1000, num_workers = 0)

seqs, p_unpaired, reactivities = df_toArray(X_DMS_test_seq, X_DMS_test_p_unpaired, y_DMS_test)
test_dataset = BedPeaksDataset(seqs, p_unpaired, reactivities)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, num_workers = 0)

In [ ]:
import timeit
my_cnn1d_1 = CNN_1d()
# print(my_cnn1d.seq_le
my_cnn1d_1 = my_cnn1d_1.float()
my_cnn1d_1, train_losses, test_losses = train_model(my_cnn1d_1, train_dataloader, validation_dataloader) # lr = 0.001 weight_decay = 0

In [ ]:
device = 'cuda'
outputs = []
expected = []
for (x,y) in test_dataloader: # iterate over batches
    # print(y)
    if torch.cuda.is_available():
        x = x.to(device)
    output = my_cnn1d_1(x).squeeze() # your awesome model here!
    #output = torch.sigmoid(output)
    output_np = output.detach().cpu().numpy()
    outputs.append(output_np)
    expected.append(y.numpy())
output_np = np.concatenate(outputs)
expected_np = np.concatenate(expected)

print(output_np)
print(expected_np)

output_np = output_np.clip(0, 1)
expected_np = expected_np.clip(0,1)

mae1 = np.mean(np.abs(output_np - expected_np))
mae1

In [ ]:
# Save the model
model_name = "CNN_3_DMS.pth"
torch.save(my_cnn1d_1.state_dict(), path_to_assignment_dir / model_name)

In [ ]:
# Plot the results
import matplotlib.pyplot as plt
plt.figure(figsize = (12,8))
plt.plot(train_losses, label="train")
plt.plot(test_losses, label="validation")
plt.title('Training & Validation Loss (DMS) CNN Model 1', fontsize = 24)
plt.xlabel('Epochs', labelpad = 15, fontsize = 16)
plt.ylabel('Loss', labelpad = 15, fontsize = 16)
plt.legend()
plt.grid(which="both")
plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fe343e39-d2c0-4296-915d-091d9a42752d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>